In [2]:
import sys
import numpy as np
import pandas as pd
from tdc.single_pred import ADME
from sklearn.preprocessing import MinMaxScaler
from jazzy.api import molecular_vector_from_smiles as mol_vect
from project_resources.import_utils import NotebookFinder
sys.meta_path.append(NotebookFinder())
from project_resources.cytochrome_P450 import load_TDC_data, min_max_scaling

importing Jupyter notebook from C:\Users\Lukas\Documents\datacytochromy\project_resources\cytochrome_P450.ipynb


In [9]:
# create csv files with Jazzy features if not already created
for benchmark in ["obach", "microsome", "hepatocyte"]:
    smi_train, target_train, smi_val, target_val, smi_test, target_test = load_TDC_data(benchmark)
    print(smi_train.shape, target_train.shape, smi_val.shape, target_val.shape, smi_test.shape, target_test.shape)
    
    y_train, y_val, y_test = min_max_scaling(target_train, target_val, target_test)
    
    ml_data_splits = {"train": (smi_train, y_train), "valid": (smi_val, y_val), "test": (smi_test, y_test)}

    for split in ["train", "valid", "test"]:
        try:
            df = pd.read_csv(f"project_resources/jazzy_splits/{benchmark}_{split}.csv")
            print(f"{benchmark}_{split}.csv already exists in project_resources/jazzy_splits")

        except FileNotFoundError:
            # generate Jazzy features and save to csv files
            jazzy_fps = []
            jazzy_ys = []

            smi_split, y_split = ml_data_splits[split]
            for smi, y in zip(smi_split, y_split):
                try:
                    jazzy_fp = mol_vect(smi)
                except:
                    jazzy_fp = None
                if jazzy_fp and not np.isnan(np.array(list(jazzy_fp.values()))).any():
                    jazzy_fp_list = np.array([fp for fp in jazzy_fp.values()])
                    jazzy_fps.append(jazzy_fp_list)
                    jazzy_ys.append(y)

            print(np.array(jazzy_fps).shape, np.array(jazzy_ys).shape)

            jazzy_csv = f"project_resources/jazzy_splits/{benchmark}_{split}.csv"
            df = pd.DataFrame(jazzy_fps, columns=['sdc', 'sdx', 'sa', 'dga', 'dgp', 'dgtot'])
            df.insert(0, "Y", jazzy_ys)
            df.to_csv(jazzy_csv, index=False)
            print(f"{jazzy_csv} was successfully created")

Found local copy...
Loading...
Done!
100%|███████████████████████████████████████████████████████████████████████████████| 667/667 [00:00<00:00, 817.47it/s]
Found local copy...
Loading...
Done!


(466,) (466,) (66,) (66,) (135,) (135,)
obach_train.csv already exists in project_resources/jazzy_splits
obach_valid.csv already exists in project_resources/jazzy_splits
obach_test.csv already exists in project_resources/jazzy_splits


100%|█████████████████████████████████████████████████████████████████████████████| 1102/1102 [00:01<00:00, 861.62it/s]
Found local copy...
Loading...
Done!


(771,) (771,) (110,) (110,) (221,) (221,)
microsome_train.csv already exists in project_resources/jazzy_splits
microsome_valid.csv already exists in project_resources/jazzy_splits
microsome_test.csv already exists in project_resources/jazzy_splits


100%|█████████████████████████████████████████████████████████████████████████████| 1213/1213 [00:01<00:00, 858.99it/s]


(849,) (849,) (121,) (121,) (243,) (243,)
hepatocyte_train.csv already exists in project_resources/jazzy_splits
hepatocyte_valid.csv already exists in project_resources/jazzy_splits
hepatocyte_test.csv already exists in project_resources/jazzy_splits
